# CAPSTONE PROJECT

### Importing Necessary Libraries

In [24]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Obtaining the Data¶

Firstly, we need a list of neighbourhoods of Toronto. For the Toronto neighborhood data, a Wikipedia page exists :https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
BeautifulSoup and requests packages will be used for website scrapping. 

In [25]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
neighborhoods_data = soup.find('table',{'class':'wikitable sortable'})
neighborhoods_data_str = str(neighborhoods_data)
df = pd.read_html(neighborhoods_data_str)[0]
df = df.rename(columns = {'Postcode':'PostalCode','Neighbourhood':'Neighborhood'})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Cleaning

There should be done 3 corrections in the dataset. 
1. In Brough column, the rows which are Not assigned will be removed.
2. Postal code areas which have more than one neighbourhood will be combined into one row.
3. In Neighbourhood column, the rows which are Not assigned will be changed to the brough name.

In [26]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#More than one neighborhood can exist in one postal code area. 
df1 = df.groupby("PostalCode", as_index=False).agg(lambda x:','.join(set(x)))
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
df1.loc[df1['Neighborhood']=="Not assigned",'Neighborhood']=df1.loc[df1['Neighborhood']=="Not assigned",'Borough']
df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
df1.shape

(103, 3)

The dataset has 103 rows and 3 columns.

In order to get venue data and plot a map, the coordinates of each neighnourhood is required. It is obtained from a csv file :http://cocl.us/Geospatial_data

In [28]:
# Read a file that includes coordinates of postal codes 
coordinates = pd.read_csv("/home/lpy/Downloads/Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
# Adding 'Latitude' and 'Longitude'columns to our dataframe
df1['Latitude']=coordinates['Latitude'].values
df1['Longitude']=coordinates['Longitude'].values
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Since the project focuses on the Toronto only, the data will be filtered.

In [30]:
df2 = df1[df1['Borough'].str.contains("Toronto")].reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Exploratory Data Analysis

#### Neighbourhoods of Toronto Map

geopy library is used to get the latitude and longitude values of Toronto.

In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [32]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Explore Neighborhoods in Toronto

Foursquare API will be used to explore venues in the neighbourhoods in Toronto. Top 100 venues which are within a radius of 1000 meters will be requested from Foursquare API.

In [47]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'ULJAWACSBIL41ZFLQZJR4EO4P3KDHXFBIMGWHPVBSZIRC5L1' # your Foursquare ID
CLIENT_SECRET = 'SSZVJPBVK4FVV55M4WL42QWVOTPPKPGCHYIX0F0HCQVKS01L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [48]:
#Create a function that gets the information for all neighbourhoods in Toronto
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
# run the created function to get venues
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )


The Beaches
Riverdale,The Danforth West
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Rathnelly,Forest Hill SE,Summerhill West,Deer Park,South Hill
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Regent Park,Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
King,Richmond,Adelaide
Toronto Islands,Union Station,Harbourfront East
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
Yorkville,North Midtown,The Annex
University of Toronto,Harbord
Chinatown,Kensington Market,Grange Park
CN Tower,Island airport,Railway Lands,King and Spadina,South Niagara,Bathurst Quay,Harbourfront West
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Exhibition Place,Parkdale Village,Brockton
The Junction South,High Park
Roncesvalles,Parkda

The size of resulting dataframe is as follows:

In [50]:
print(toronto_venues.shape)
toronto_venues.head()

(3083, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


#### Analyze Each Neighbourhood

In [51]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns = cols)


toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [52]:
toronto_onehot.shape

(3083, 274)

Next, rows will be grouped by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [54]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Berczy Park,0.0,0.0,0.0,0.0000,0.0000,0.010000,0.0,0.0,0.0,0.0,0.02,0.00,0.000000,0.0,0.0,0.0,0.0,0.010000,0.01,0.030000,0.0,0.000000

Since Coffee Shops are the interest of this project, the dataframe will be filtered to coffee shops only.

In [59]:
cof_shop = toronto_grouped[["Neighborhood","Coffee Shop"]]
cof_shop.head()

,Neighborhood,Coffee Shop
0,Berczy Park,0.070000
1,Business Reply Mail Processing Centre 969 Eastern,0.083333
2,"CN Tower,Island airport,Railway Lands,King and...",0.187500
3,"Cabbagetown,St. James Town",0.052632
4,Central Bay Street,0.080000


#### Cluster Neighborhoods

k-means clustering method will be used to analyze the data. The number of clusters, which is k, is determined to be 3. 

In [61]:
# set number of clusters
kclusters = 3

cof_shop_clustering = cof_shop.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cof_shop_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 1, 2, 1, 1, 2, 2, 2], dtype=int32)

After obtaining the labels, they are added to the data.

In [62]:
# create a copy of coffee shop-neighborhood data
cof_shop_df = cof_shop.copy()
# add cluster labels
cof_shop_df["Cluster Label"] = kmeans.labels_

In [64]:
cof_shop_df.head()

,Neighborhood,Coffee Shop,Cluster Label
0,Berczy Park,0.070000,1
1,Business Reply Mail Processing Centre 969 Eastern,0.083333,2
2,"CN Tower,Island airport,Railway Lands,King and...",0.187500,0
3,"Cabbagetown,St. James Town",0.052632,1
4,Central Bay Street,0.080000,2


In [72]:
# Add latitude/longitude for each neighborhood
coffee_shop = df2.join(cof_shop_df.set_index('Neighborhood'), on='Neighborhood')

coffee_shop.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Coffee Shop,Cluster Label
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.060976,1
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,0.090000,2
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0.054054,1
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.070000,1
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.111111,2


#### Visualization of Resulting Clusters

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(coffee_shop['Latitude'], coffee_shop['Longitude'], coffee_shop['Neighborhood'], coffee_shop['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

Cluster 0

In [74]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Coffee Shop,Cluster Label
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0.130000,0
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0.130435,0
13,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,0.160000,0
27,M5V,Downtown Toronto,"CN Tower,Island airport,Railway Lands,King and...",43.628947,-79.394420,0.187500,0


In [78]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 0]['Coffee Shop'].mean()

0.15198369565217393

Cluster 1

In [75]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Coffee Shop,Cluster Label
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.060976,1
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0.054054,1
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.070000,1
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0.068182,1
11,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0.052632,1
14,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.060000,1
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.060000,1
16,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.070000,1
18,M5H,Downtown Toronto,"King,Richmond,Adelaide",43.650571,-79.384568,0.050000,1
19,M5J,Downtown Toronto,"Toronto Islands,Union Station,Harbourfront East",43.640816,-79.381752,0.070000,1


In [79]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 1]['Coffee Shop'].mean()

0.05636829606675826

Cluster 2

In [76]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Coffee Shop,Cluster Label
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,0.090000,2
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.111111,2
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0.090000,2
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,0.084746,2
9,M4V,Central Toronto,"Rathnelly,Forest Hill SE,Summerhill West,Deer ...",43.686412,-79.400049,0.103896,2
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.080000,2
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.080000,2
21,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0.080000,2
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0.074074,2
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0.090000,2


In [80]:
coffee_shop.loc[coffee_shop['Cluster Label'] == 2]['Coffee Shop'].mean()

0.08723254847880968

According to the cluster averages, Cluster 1 has the minimum frequency of occurrence of Coffee Shop. So that it is reasonable to make further exploration on those neighborhoods with the label cluster 1 since merket competition is low in those areas.

#### Examine Other Venues in Selected Cluster (Cluster 1)

In [83]:
# make a new dataframe called target that includes cluster 1 data
target = coffee_shop.loc[coffee_shop['Cluster Label'] == 1]
target.shape

(21, 7)

In [98]:
test =toronto_grouped.loc[toronto_grouped['Neighborhood'].isin(target['Neighborhood'])]
#target_neighborhood=toronto_grouped[toronto_grouped['Neighborhood']==target['Neighborhood']]
test.shape

(21, 274)